In [ ]:
import numpy as np
from ian.ian import *
from ian.dset_utils import *
from ian.utils import *
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

plt.rcParams['figure.figsize']  = (6.0, 4.0)
plt.rcParams['figure.dpi'] = 72.0

In [ ]:
dset = dset_stingray()
data_cmap = None

X = dset['X']
c = dset['c']
N = X.shape[0]

_ = plot2dScatter(X, c, s=25, cmap=data_cmap)

In [ ]:
D2 = getD2(X) #square matrix of squared Euclidean distances
Xplot = X
G, wG, optScales, disc_pts = IAN('exact-precomputed-sq', D2, Xplot=Xplot, plot_interval=1)

### Visualizing resulting scales and data graphs

In [ ]:
f,axes = subps(1,3,4,4)

### INDIVIDUAL SCALES
ax = axes[0]
ax.set_title('Individual scales')

lvl = .75 #level set of Gaussian kernel to plot

# use kernel formula and solve for the distance, d (radius) using the optimal individual scale, s: 
# lvl = exp(-d^2/(s^2)) --> sˆ2 log(1/lvl) = d^2 --> d = s sqrt(log(1/lvl))

scales_radii = optScales*np.sqrt(np.log(1/lvl))

plotScales(Xplot, [], scales_radii, scalesScl=1, f_axes=(f,[ax]), circleColor='g')

### UNWEIGHTED GRAPH

ax = axes[1]
ax.set_title('Unweighted graph, $G$')

plotDataGraph(Xplot, G, f_ax=(f,ax), edge_color='k', edge_width=.75)


### WEIGHTED GRAPH

ax = axes[2]
ax.set_title('Weighted graph, $\mathcal{G}^{\star}$')
#normalize weights for better visualization and comparison with other methods
#divide by kernel value when the scale equals the distance
#exp( -d^2 / s^2 ) = exp(-1) when d == s
wAdj = wG.toarray()
wAdj /= np.exp(-1)
wAdj[wAdj > 1] = 1 #cant have opacity larger than 1
plotWeightedGraph(Xplot, wAdj, rgb=(0,0,0), f_ax=(f,ax))

plt.show()




### Local dimensionality

In [ ]:
#run NCD algorithm to estimate local dimension
nbrhoodOrder = 3 #using neighbors-of-neighbors up to 2 hops away
NofNDims, degDims = estimateDims(G, D2, nbrhoodOrder) 
dims = np.maximum(degDims,NofNDims)


#set bounds for colormap
MIND, MAXD = min(0.5,dims.min()),  max(dims.max(),3.)
norm = matplotlib.colors.Normalize(vmin=MIND, vmax=MAXD)
dim_cmap = matplotlib.cm.get_cmap('gist_ncar')

dimColors = [dim_cmap(norm(si)) for si in dims]

f,ax = subps(1,1,4,4)

plot2dScatter(X, dimColors, f_ax=(f,ax), axLabels=False, s=25)

ax = f.add_axes([0.05, -.025, 0.9, 0.025])
import matplotlib as mpl

cb = mpl.colorbar.ColorbarBase(ax, orientation='horizontal', 
                               cmap=dim_cmap,
                               norm=mpl.colors.Normalize(MIND, MAXD),
                               label='local dimension')

plt.show()

f,ax = subps(1,1,4,4)
ax.hist(dims)
ax.set_title('Distribution of local dimension')
plt.show()

### Compare embeddings using different algorithms

In [ ]:
D1 = np.sqrt(D2) # (non-sq) distances

### IAN + ISOMAP
f,ax = subps(1,1,4,4)
ax.set_title('IAN + Isomap')

dwG = G.multiply(D1) # elementwise multiply to get discrete graph using distances as weights
n_components = 2
_, isomap_y = computeIsomap(X, None, n_components, knbrs_graph=dwG)
plot2dScatter(isomap_y, c, cmap=data_cmap, f_ax=(f,ax), s=20, axLabels=False)
plt.show()

In [ ]:
### IAN + Diffusion Maps

%matplotlib notebook
f,ax = subps(1,1,6,6,d3=True)
ax.set_title('IAN + Diffusion Maps')
alpha = 1
t = 1
n_components = 3
diffmap_y, _ = diffusionMapFromK(wG, n_components, alpha, t)

plot3dScatter(diffmap_y,None,f_ax=(f,ax),s=20,depthshade=False,axisEqual=True,noTicks=False)
plt.show()

### Heat geodesics

In [ ]:
geocenter = [116] #index of starting node(s) for heat diffusion

K = wG.toarray() #convert weighted graph to dense array

#remove any self-weights
np.fill_diagonal(K,0)

t = 10 #heat diffusion time
phis = computeHeatGeodesics(K, t, geocenter)


f,ax = subps(1,1,4,4)

zorder = phis.argsort()[::-1]#put hottest pts on top for clearer visualization
plot2dScatter(X[zorder], phis[zorder], f_ax=(f,ax), axLabels=False, cmap=plasma7r_cmp, s=100, edgecolors=(0,0,0))
plt.show()